<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_twadr_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-4.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-4.validation.csv",header=None)
valid.head()

,0,1
0,61,very stupid
1,74,useless without
2,93,shooting
3,195,makes me so damn bloated
4,231,gives me man vivid dreams


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-4.test.csv",header=None)
test.head()

,0,1
0,13,insomnia
1,13,insomnia
2,56,painful b..
3,62,hypomanic
4,84,paraplegic


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-cadec.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'i',
 'the',
 'and',
 'to',
 'my',
 'pain',
 'of',
 'in',
 'a',
 'it',
 'me',
 'mood',
 'increased',
 'for',
 'have',
 '-',
 'was',
 'drug',
 ')',
 'lipitor',
 '(',
 'this',
 'on',
 'disorder',
 'abnormal',
 'not',
 'had',
 'withdrawal',
 'that',
 'symptoms',
 'anxiety',
 'is',
 'with',
 'muscle',
 'weight',
 'like',
 'but',
 'decreased',
 'feeling',
 "n't",
 'taking',
 'disorders',
 'sleeplessness',
 'up',
 'after',
 'syndrome',
 'behavior',
 'sleep',
 'blood',
 'so',
 'diseases',
 'insomnia',
 'got',
 'am',
 'feel',
 'all',
 'side',
 'energy',
 'depression',
 'somnolence',
 'finding',
 'from',
 'disturbance',
 'effects',
 '!',
 'back',
 'loss',
 'be',
 'been',
 'mental',
 'day',
 'heart',
 'as',
 'skin',
 'cholesterol',
 'body',
 'gain',
 'take',
 'appetite',
 'acute',
 'no',
 '/',
 'nausea',
 'at',
 'now',
 'off',
 'do',
 'has',
 'fatigue',
 'about',
 'mg',
 'years',
 'dizziness',


In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.306536,3.457965,0.431822,09:05


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.267616,3.026124,0.489174,08:55


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.059953,2.990225,0.493471,09:01


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.006375,2.961155,0.496568,09:00


In [25]:
learn.save_encoder('TwADR-cadec_fold4_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-cadec_fold4_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.147584,6.727969,0.058333,00:25


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.202625,5.842129,0.125000,00:26


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.692034,5.014335,0.150000,01:09


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.402799,4.736977,0.175000,01:03
1,5.141881,4.502251,0.216667,01:08


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.941268,4.295733,0.233333,01:07
1,4.728354,4.200907,0.250000,01:06


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.519309,3.962932,0.266667,01:08
1,4.341138,3.894563,0.283333,01:03


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.142485,3.769136,0.300000,01:07
1,4.001356,3.761849,0.283333,01:06


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.813526,3.645154,0.325000,01:04
1,3.691638,3.588827,0.325000,01:02


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.473383,3.490860,0.366667,01:05
1,3.420986,3.559900,0.300000,01:05
2,3.264878,3.439771,0.366667,01:08
3,3.101749,3.452209,0.391667,01:09


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.961788,3.459029,0.333333,01:05
1,2.878929,3.455432,0.333333,01:07
2,2.720313,3.441921,0.358333,01:10
3,2.577511,3.458439,0.358333,01:05


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.437379,3.460439,0.375000,01:03
1,2.379917,3.456452,0.375000,01:05
2,2.215513,3.362992,0.416667,01:11
3,2.064884,3.402957,0.375000,01:09


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.922244,3.413975,0.383333,01:06
1,1.895684,3.562439,0.358333,01:10
2,1.772988,3.512934,0.416667,01:07
3,1.645582,3.569577,0.400000,01:05


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.564221,3.541104,0.391667,01:08
1,1.538525,3.628438,0.375000,01:03
2,1.404828,3.657474,0.400000,01:04
3,1.330609,3.633701,0.400000,01:07


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.174394,3.614529,0.400000,01:04
1,1.195283,3.794473,0.391667,01:08
2,1.096752,3.703133,0.391667,01:05
3,0.993668,3.614974,0.391667,01:08


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.922903,3.792342,0.358333,01:09
1,0.962329,3.740606,0.375000,01:12
2,0.923125,3.787871,0.391667,01:12
3,0.834559,3.793742,0.400000,01:08


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.741129,3.725758,0.408333,01:08
1,0.724974,3.821234,0.391667,01:09


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.719873,3.919568,0.383333,01:06
1,0.703193,3.937141,0.383333,01:04


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.694999,3.869509,0.391667,01:03


In [44]:
# save the best model

learn.save_encoder('TwADR-cadec_fold4')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.634884,3.935308,0.400000,01:09


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.596203,4.057834,0.383333,01:03


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.584107,3.891435,0.391667,01:07


# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1787, 1787, 720, 2024, 1595, 109, 189, 189, 189, 230, 231, 1936, 233, 233, 2032, 1787, 1136, 908, 2149, 2063, 2002, 187, 1769, 596, 1290, 1432, 749, 749, 97, 1597, 792, 1484, 1458, 1458, 2147, 824, 908, 908, 923, 1278, 824, 1718, 974, 978, 982, 978, 978, 978, 978, 418, 1828, 1089, 1711, 1099, 1099, 1099, 1769, 1136, 1136, 1145, 1150, 1150, 1308, 1184, 1184, 1718, 1204, 974, 24, 1265, 857, 1099, 1308, 824, 1529, 2149, 1099, 1936, 1099, 1508, 1508, 1511, 1526, 1529, 1711, 1682, 1594, 1595, 791, 60, 1597, 1099, 1099, 1718, 1718, 1718, 1610, 978, 1073, 1073, 1787, 1680, 1701, 1526, 418, 56, 1718, 1718, 1073, 1815, 1718, 1815, 1578, 2191, 2191, 2149, 2023, 1857, 1769, 825, 824, 1936, 1936, 1727, 2002, 1899, 2026, 2032, 2032, 189, 1508, 1099, 791, 2103, 2014, 2119, 2147, 966, 2032, 2147, 1936, 1711, 2149]
[13, 13, 56, 62, 84, 109, 189, 189, 189, 230, 231, 233, 233, 233, 393, 393, 396, 418, 418, 452, 452, 483, 561, 596, 712, 729, 749, 749, 750, 791, 792, 792, 809, 809, 813, 824, 908,

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

60
0.4195804195804196
